In [56]:
!pip install gcloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gcloud: filename=gcloud-0.18.3-py3-none-any.whl size=602926 sha256=023bf1420ea13cf5146feadb811936106fb91d64ff2e467351ae6210922d4e65
  Stored in directory: /root/.cache/pip/wheels/7c/30/88/5017af921da3a33af785f0d0fd3e944b845bc62a445a2c2f69
Successfully built gcloud


In [57]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=KterMdGzLxXR58whEjLBpZQRPnippY&prompt=consent&token_usage=remote&access_type=offline&code_challenge=Ij9VvcTeunqj9BYu5rlNqyrhfIfPdNeQ6599VmKDhzE&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AQlEd8zY2CaqZTnDjfoeYT2nzQxt6Yz26fn7G6nEizrgn4sE7Y_Ip7HvKPuirIckybXnjQ

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [58]:
##importando pacotes
import pandas as pd
import numpy as np
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import csv
import re

In [ ]:
#Alterar configuração para reshaped quando for extrair o csv, isso ja torna as colunas tratadas

In [59]:
df = pd.read_csv("ilostat_todos_paises_v2.csv")
#df['obs_value'] = df['obs_value'] * 1000
df

,Country,Source,Sex,Age,Year,Private,Public
0,Afghanistan,LFS - Labour Force Survey,Total,Total,2021,7036.6,642.8
1,Afghanistan,LFS - Labour Force Survey,Male,Total,2021,5258.7,591.8
2,Afghanistan,LFS - Labour Force Survey,Female,Total,2021,1778.0,51.1
3,Afghanistan,LFS - Labour Force Survey,Total,Total,2020,5546.5,532.0
4,Afghanistan,LFS - Labour Force Survey,Male,Total,2020,4465.4,472.4
...,...,...,...,...,...,...,...
2347,Zimbabwe,LFS - Labour Force Survey,Male,Total,2019,2271.2,179.1
2348,Zimbabwe,LFS - Labour Force Survey,Female,Total,2019,2316.3,170.7
2349,Zimbabwe,LFS - Labour Force Survey,Total,Total,2014,5921.6,343.6
2350,Zimbabwe,LFS - Labour Force Survey,Male,Total,2014,2888.0,203.0


In [60]:
df = df.rename(columns={
    'Country':'Reference area',
    'Year':'Time'
})

df

,Reference area,Source,Sex,Age,Time,Private,Public
0,Afghanistan,LFS - Labour Force Survey,Total,Total,2021,7036.6,642.8
1,Afghanistan,LFS - Labour Force Survey,Male,Total,2021,5258.7,591.8
2,Afghanistan,LFS - Labour Force Survey,Female,Total,2021,1778.0,51.1
3,Afghanistan,LFS - Labour Force Survey,Total,Total,2020,5546.5,532.0
4,Afghanistan,LFS - Labour Force Survey,Male,Total,2020,4465.4,472.4
...,...,...,...,...,...,...,...
2347,Zimbabwe,LFS - Labour Force Survey,Male,Total,2019,2271.2,179.1
2348,Zimbabwe,LFS - Labour Force Survey,Female,Total,2019,2316.3,170.7
2349,Zimbabwe,LFS - Labour Force Survey,Total,Total,2014,5921.6,343.6
2350,Zimbabwe,LFS - Labour Force Survey,Male,Total,2014,2888.0,203.0


In [61]:
df.columns

Index(['Reference area', 'Source', 'Sex', 'Age', 'Time', 'Private', 'Public'], dtype='object')

In [62]:
## Passar para long

df_long = df.melt(id_vars =['Reference area', 'Source','Time','Sex'],
        value_vars =['Public', 'Private'],
        var_name ='Setor',
        value_name ='Total')
df_long

,Reference area,Source,Time,Sex,Setor,Total
0,Afghanistan,LFS - Labour Force Survey,2021,Total,Public,642.8
1,Afghanistan,LFS - Labour Force Survey,2021,Male,Public,591.8
2,Afghanistan,LFS - Labour Force Survey,2021,Female,Public,51.1
3,Afghanistan,LFS - Labour Force Survey,2020,Total,Public,532.0
4,Afghanistan,LFS - Labour Force Survey,2020,Male,Public,472.4
...,...,...,...,...,...,...
4699,Zimbabwe,LFS - Labour Force Survey,2019,Male,Private,2271.2
4700,Zimbabwe,LFS - Labour Force Survey,2019,Female,Private,2316.3
4701,Zimbabwe,LFS - Labour Force Survey,2014,Total,Private,5921.6
4702,Zimbabwe,LFS - Labour Force Survey,2014,Male,Private,2888.0


In [63]:
## fazer o total

df = df_long[df_long['Sex']=="Total"]


In [64]:
df

,Reference area,Source,Time,Sex,Setor,Total
0,Afghanistan,LFS - Labour Force Survey,2021,Total,Public,642.8
3,Afghanistan,LFS - Labour Force Survey,2020,Total,Public,532.0
6,Afghanistan,HIES - Households Living Conditions Survey,2017,Total,Public,648.8
9,Afghanistan,HIES - Households Living Conditions Survey,2014,Total,Public,722.5
12,Angola,LFS - Employment Survey,2021,Total,Public,948.3
...,...,...,...,...,...,...
4689,Zambia,HIES - Living Conditions Monitoring Survey,2015,Total,Private,3317.7
4692,Zimbabwe,LFS - Labour Force Survey,2022,Total,Private,2580.8
4695,Zimbabwe,LFS - Labour Force Survey,2021,Total,Private,2728.0
4698,Zimbabwe,LFS - Labour Force Survey,2019,Total,Private,4587.5


In [65]:
df.columns

Index(['Reference area', 'Source', 'Time', 'Sex', 'Setor', 'Total'], dtype='object')

In [66]:
x=df[['Reference area', 'Source', 'Time', 'Setor', 'Total']].pivot_table(index=['Reference area','Time' ], values='Total',aggfunc=np.sum)
y = pd.DataFrame(x)
total = y.reset_index()

<ipython-input-66-c6c490a86459>:1: FutureWarning: The provided callable <function sum at 0x7b409faf6e60> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  x=df[['Reference area', 'Source', 'Time', 'Setor', 'Total']].pivot_table(index=['Reference area','Time' ], values='Total',aggfunc=np.sum)


In [67]:
df= df[['Reference area', 'Source', 'Time', 'Setor', 'Total']]
df= df.rename(columns={"Total":"quantidade_vinculos"})

In [68]:
df1= df.merge(total,right_on=['Reference area','Time'],left_on=['Reference area','Time'])

In [69]:
df1['prop'] = df1['quantidade_vinculos']/df1['Total']

In [70]:
df1['prop'] = np.where(df1['prop']==1,np.nan,df1['prop'])

In [71]:
df1[df1['Reference area']=="Brazil"]

,Reference area,Source,Time,Setor,quantidade_vinculos,Total,prop
200,Brazil,HS - Continuous National Household Sample Survey,2023,Public,12108.3,99281.8,0.121959
201,Brazil,HS - Continuous National Household Sample Survey,2023,Private,87173.5,99281.8,0.878041
202,Brazil,HS - Continuous National Household Sample Survey,2022,Public,11852.7,97919.0,0.121046
203,Brazil,HS - Continuous National Household Sample Survey,2022,Private,86066.3,97919.0,0.878954
204,Brazil,HS - Continuous National Household Sample Survey,2021,Public,11352.2,91187.7,0.124493
205,Brazil,HS - Continuous National Household Sample Survey,2021,Private,79835.5,91187.7,0.875507
206,Brazil,HS - Continuous National Household Sample Survey,2020,Public,11609.1,86874.7,0.133630
207,Brazil,HS - Continuous National Household Sample Survey,2020,Private,75265.6,86874.7,0.866370
208,Brazil,HS - Continuous National Household Sample Survey,2019,Public,11366.2,94129.0,0.120751
209,Brazil,HS - Continuous National Household Sample Survey,2019,Private,82762.8,94129.0,0.879249


In [ ]:
## Tratamento adicional:

#Quando a prop for 1.0 ignorar

In [72]:
paises = ["Japan","United Kingdom","Canada","Australia","Germany"]

In [73]:
selecionados = df1[df1['Reference area'].isin(paises)]

In [74]:
df1= df1.rename(columns={'Time':'ano','Reference area':'pais','Source':'fonte_pesquisa','Total':'total_ano','Setor':'setor'})

In [75]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1568 entries, 0 to 1567
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pais                 1568 non-null   object 
 1   fonte_pesquisa       1568 non-null   object 
 2   ano                  1568 non-null   int64  
 3   setor                1568 non-null   object 
 4   quantidade_vinculos  1568 non-null   float64
 5   total_ano            1568 non-null   float64
 6   prop                 1567 non-null   float64
dtypes: float64(3), int64(1), object(3)
memory usage: 85.9+ KB


In [76]:
linhas_sem_prop = df1[df1['prop'].isna()]
print(linhas_sem_prop)

      pais                                  fonte_pesquisa   ano    setor  \
1051  Niue  HIES - Household Income and Expenditure Survey  2015  Private   

      quantidade_vinculos  total_ano  prop  
1051                  0.9        0.9   NaN  


In [77]:
df1.loc[[1050, 1051]]


,pais,fonte_pesquisa,ano,setor,quantidade_vinculos,total_ano,prop
1050,Niue,HIES - Household Income and Expenditure Survey,2015,Public,0.0,0.9,0.0
1051,Niue,HIES - Household Income and Expenditure Survey,2015,Private,0.9,0.9,NaN


In [78]:
df1 = df1.drop(index=[1050, 1051])

In [79]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1566 entries, 0 to 1567
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pais                 1566 non-null   object 
 1   fonte_pesquisa       1566 non-null   object 
 2   ano                  1566 non-null   int64  
 3   setor                1566 non-null   object 
 4   quantidade_vinculos  1566 non-null   float64
 5   total_ano            1566 non-null   float64
 6   prop                 1566 non-null   float64
dtypes: float64(3), int64(1), object(3)
memory usage: 97.9+ KB


In [80]:
min(df1['ano'])

2013

In [81]:
schema=[
 bigquery.SchemaField('pais','STRING',description='Nome do país selecionado'),
 bigquery.SchemaField('fonte_pesquisa','STRING',description='De qual pesquisa nacional foi extraído aquele dado.'),
bigquery.SchemaField('ano','Integer',description='Ano de coleta da informação'),
bigquery.SchemaField('setor','STRING',description='se o setor é público ou privado'),
bigquery.SchemaField('quantidade_vinculos','float',description='total de pessoas que trabalham naquele setor'),
bigquery.SchemaField('total_ano','float',description='total de pessoas que trabalham naquele ano'),
 bigquery.SchemaField('prop','Float',description='Proporção de vínculos naquele setor'),

 ]

In [82]:
## Subindo para datalake
client = bigquery.Client(project='repositoriodedadosgpsp')
dataset_ref = client.dataset('perfil_remuneracao')

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [83]:
table_ref = dataset_ref.table('ILOSTAT_todos_paises_v2') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df1, table_ref, job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=35ec81a8-c356-4fd9-94ea-8fec8dd0b1cd>